In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing import image

# Ensure TensorFlow and Keras versions
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")

# Load the ResNet50 model without the top (fully connected layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

TensorFlow version: 2.16.1
Keras version: 3.3.3


In [7]:
# Freeze the ResNet50 layers so they are not trained
for layer in base_model.layers:
    layer.trainable = False

In [8]:

# Add custom classification layers
model = Sequential([
    Input(shape=(224, 224, 3)),  # Explicitly add an Input layer
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: image1, image2, image3
])

# Learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(scheduler)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2]
)

# Data generator for reading and augmenting data from directories
train_generator = train_datagen.flow_from_directory(
    r"C:\Users\hp\Documents\Miniproject\trail\train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Data generator for validation/test set (only rescaling)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    r"C:\Users\hp\Documents\Miniproject\trail\test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 7249 images belonging to 3 classes.
Found 1858 images belonging to 3 classes.


In [10]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[lr_scheduler]
)


Epoch 1/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 665s 3s/step - accuracy: 0.5618 - loss: 1.2009 - val_accuracy: 0.8385 - val_loss: 0.4893 - learning_rate: 1.0000e-04
Epoch 2/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 656s 3s/step - accuracy: 0.7551 - loss: 0.6062 - val_accuracy: 0.8149 - val_loss: 0.5426 - learning_rate: 1.0000e-04
Epoch 3/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 645s 3s/step - accuracy: 0.8283 - loss: 0.4809 - val_accuracy: 0.8245 - val_loss: 0.4266 - learning_rate: 1.0000e-04
Epoch 4/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 643s 3s/step - accuracy: 0.8394 - loss: 0.4272 - val_accuracy: 0.8337 - val_loss: 0.4804 - learning_rate: 1.0000e-04
Epoch 5/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 646s 3s/step - accuracy: 0.8759 - loss: 0.3619 - val_accuracy: 0.8945 - val_loss: 0.2758 - learning_rate: 1.0000e-04
Epoch 6/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 644s 3s/step - accuracy: 0.8980 - loss: 0.3046 - val_accuracy: 0.9015 - val_loss: 0.2875 - learning_rate: 1.0000e-05
Epoch 7/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 645s 3s/step - acc

In [11]:
# Prediction
# Predict classes for test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Map predictions to class labels
class_names = list(train_generator.class_indices.keys())
predicted_labels = [class_names[i] for i in predicted_classes]

# Print the predictions
print(predicted_labels)


59/59 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step
['class1', 'class2', 'class1', 'class1', 'class1', 'class1', 'class2', 'class2', 'class2', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class3', 'class1', 'class1', 'class1', 'class1', 'class2', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class2', 'class1', 'class2', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class2', 'class1', 'class1', 'class1', 'class2', 'class1', 'class1', 'class2', 'class1', 'class2', 'class3', 'class1', 'class1', 'class1', 'class1', 'class2', 'class2', 'class1', 'class1', 'class1', 'class2', 'class2', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class2', 'class1', 'class1', 'class1', 'class1', 'class1', 'class2', 'class2', 'class2', 'class1', 'class1', 'class2', 'class1', 'class1', 'class2', 'class1', 'class2', 'class1', 'class1', 'class1', 'class2', 'class2', 'class3', 'class2', 'class1',

In [12]:
# Save the model
model.save('resnet50_image_classifier.h5')
print("Model saved successfully.")

Model saved successfully.


In [27]:
# 1. Accuracy and Loss Curves
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('accuracy_loss_curves.png')
plt.close()

In [33]:
# 2. Confusion Matrix
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
class_names = list(train_generator.class_indices.keys())
cm = confusion_matrix(test_generator.classes, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix.png')
plt.close()
plt.show()

59/59 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step


In [29]:
# 3. Classification Report
print("\nClassification Report:")
print(classification_report(test_generator.classes, y_pred, target_names=class_names))


Classification Report:
              precision    recall  f1-score   support

      class1       0.98      0.68      0.80       588
      class2       0.76      0.97      0.85       574
      class3       0.96      0.99      0.97       696

    accuracy                           0.88      1858
   macro avg       0.90      0.88      0.87      1858
weighted avg       0.90      0.88      0.88      1858



In [30]:
# 4. Model Summary
print("\nModel Summary:")
model.summary()


Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │      25,690,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             771 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 100,661,131 (383.99 MB)

 Trainable params: 25,691,139 (98.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 51,382,280 (196.01 MB)

In [35]:


# Assuming model is already defined and loaded
# Correcting the layer selection to point to the first convolutional layer
first_conv_layer = None
for layer in model.layers[0].layers:
    if 'conv' in layer.name:
        first_conv_layer = layer
        break

# Check if the correct layer is found
if first_conv_layer is None:
    raise ValueError("No convolutional layer found in the model.")

# Define the feature map model
feature_maps = tf.keras.models.Model(inputs=model.layers[0].inputs, outputs=first_conv_layer.output)

# Load and preprocess the image
img = next(test_generator)[0][0]
img = np.expand_dims(img, axis=0)

# Get the feature map
feature_map = feature_maps.predict(img)

# Check the shape of the feature map
print("Feature map shape:", feature_map.shape)  # Debugging line

# Plot the feature maps
num_filters = feature_map.shape[-1]
fig, axes = plt.subplots(4, 8, figsize=(20, 10))
for i, ax in enumerate(axes.flat):
    if i < num_filters:  # Ensure we don't index out of bounds
        ax.imshow(feature_map[0, :, :, i], cmap='viridis')
    ax.axis('off')
plt.suptitle("Feature Maps of First Convolutional Layer")
plt.savefig('feature_maps.png')
plt.close()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Feature map shape: (1, 230, 230, 3)


In [36]:
# Save model performance metrics
with open('model_performance.txt', 'w') as f:
    f.write(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}\n")
    f.write(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}\n")
    f.write(f"Final Training Loss: {history.history['loss'][-1]:.4f}\n")
    f.write(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}\n")
    f.write("\nClassification Report:\n")
    f.write(classification_report(test_generator.classes, y_pred, target_names=class_names))

print("Graphs and performance metrics have been saved.")

Graphs and performance metrics have been saved.


In [37]:

# Load and preprocess a single image for prediction
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0, 1] range
    return img_array

# Predict the class of an image using the loaded model
def predict_image(model, img_path, class_indices, threshold=0.5):
    img_array = load_and_preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = prediction[0][predicted_class_index]

    if confidence < threshold:
        return "unknown", confidence

    class_labels = {v: k for k, v in class_indices.items()}
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label, confidence

# Load the saved model
loaded_model = tf.keras.models.load_model('resnet50_image_classifier.h5')


In [40]:
# Example usage
img_path = "Class_2.jpg"
predicted_class, confidence = predict_image(loaded_model, img_path, train_generator.class_indices)

if predicted_class == "unknown":
    print("The image does not belong to any of the known classes.")
else:
    print(f"The predicted class is: {predicted_class} with confidence {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step
The predicted class is: class2 with confidence 1.00
